# Workshop 3: Exploring IllustrisTNG simulations to derive observationally comparable star formation rates and metallicities

## Notebook 2: The global star forming main sequence

Created by Bryanne McDonough and Olivia Curtis
In this notebook, we will use the data we pulled from the TNG simulation in Notebook 1 to construct a global star forming main sequence (M_* - SFR) relation, and perform a ridge line fit. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy.ma as ma

### Update this directory! ###
#dirc='/path/to/directory/'
dirc='/projectnb/res-star/TNG_workshop/'

## Loading the data
First, let's load in the galaxy catalog we generated in the previous notebook.

Recall that we saved the galaxy catalog as a dictionary, so we can print its keys to see 
what data we have to work with.

In [ ]:
#loads the dictionary and prints its keys
dat=np.load(dirc+'galcat.npy', allow_pickle=True).item()
print(dat.keys())

Now, let's grab the variables that we will be working with in this notebook. 

In [3]:
#unpacking stellar mass, instanteous SFRs, and SFRs over the past 10 Myrs
mass= dat['M_*'] #Units M_*
SFR_i = dat['SFR_inst'] #Units M_\odot / yr
SFR_10 = dat['SFR_all_10'] #Units M_\odot /yr

#Feel free to unpack any other variables that you may have saved!

## Plotting the M_* - SFR relation

For now, let's just use the instantaneous SFRs.

Let's start by exploring our data. First, take the log10 of the data

In [ ]:
#take the log of mass and SFR
logmass = np.log10(mass)
logSFR = np.log10(SFR_i)

#you should get a runetime warning here, ignore it for now.

Next, get the minima, medians, and maxima of each array 

In [ ]:
#get and print the stellar mass values
min_logmass = min(logmass)
median_logmass = np.median(logmass)
max_logmass = max(logmass)

print("log10(mass) min:    ", min_logmass)
print("log10(mass) median: ", median_logmass)
print("log10(mass) max:    ", max_logmass)

Let's do the same for the instantaneous star formation rates

In [ ]:
#get and print the stellar mass values
min_logSFR = min(logSFR)
median_logSFR = np.median(logSFR)
max_logSFR = max(logSFR)

print("log10(SFR) min:    ", min_logSFR)
print("log10(SFR) median: ", median_logSFR)
print("log10(SFR) max:    ", max_logSFR)

**Notice:** Some of the galaxies in our sample have SFR=0. This is due to the minimum baryonic mass of a particle in the simulations. Thus, low amounts of gas and star formation will be unresolved. You can check the minimum baryonic mass for a given simulation in the table here: https://www.tng-project.org/data/docs/background/#sec1

In order to visualize the quenched galaxies on our diagram, you can impose a fixed specific SFR. However, let's stick to the galaxies with resolved star formation.

Now, let's make a 2D histogram of the SFR - M_* plane using matplotlib. We will be using matplotlib.pyplot.hist2d for this, which you can read about here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist2d.html

In [7]:
#If you are unsure how to use hist2d,
#then run this block of code to read its 
#documentation in Jupyter
plt.hist2d?

In [ ]:
#Call hist 2D. The final histogram, hist, will
#be a 100x100 array containing the number
#of galaxies in each bin.
hist, xedges, yedges, image = plt.hist2d(logmass[np.isfinite(logSFR)], \
                                         logSFR[np.isfinite(logSFR)], \
                                         bins=100, cmin=1)

print("The shape of hist is: ", hist.shape)

#Add a vertical colorbar to the figure and label it
cbar=plt.colorbar()
cbar.set_label('$N_{\mathrm{galaxies}}$', rotation=90, fontsize=12)

#Label the axes
plt.xlabel('log($M_*$ [$M_\odot$])',fontsize=12)
plt.ylabel('log(SFR [$M_\odot$/yr])',fontsize=12)

#Change the style of the tick marks
plt.tick_params(axis='both', direction='in', \
                width=1.25, labelsize='medium', \
                which='both', top=True, right=True)

## Performing a ridge line fit

The most common main sequence fit uses ordinary least squares to fit a line to all the data. But the main sequence isn't exactly linear, and different samples can bias the slope that is obtained. Renzini & Peng (2015) argue that a more objective definition is a so-called 'ridge line' fit: https://ui.adsabs.harvard.edu/abs/2015ApJ...801L..29R/abstract. Functionally, this means we will fit to the mode of the data, or the pixel with the highest count in each mass bin.

That is, for each column in the above figure, we want to find the row that contains the most galaxies. Keep in mind that hist is a 100x100 array, so we will ultimately be saving the midpoints of rows and columns.

Start by instantiating two empty arrays.
The variable fMs will hold the midpoint of each column, 
and fSFR will hold the midpoint of the row that has the most number of galaxies in each column.

In [9]:
#instantiate two empty arrays that we will populate later
#recall that hist is a size 100x100 array.
fMs=np.empty(100)
fSFR=np.empty(100)

Next, let's remove any cells from hist that did not contain any galaxies (i.e., any cells whose value is NaN). 

In [10]:
#find all NaN cells in hist
mask=np.isnan(hist)
#mask out the invalid cells
goodhist=ma.masked_array(hist, mask=mask)

Let's print hist and goodhist to verify the mask did what we want.

In [ ]:
print(hist)
print("\n")
print(goodhist)

Notice that goodhist is still a 100x100 array

In [ ]:
print(goodhist.shape)

Next, we will loop over each column in the histogram. We will find the pixel in each column that contains the most number of galaxies, saving the midpoints of the pixel into fMs and fSFR.

We will thus find the (M_s,SFR) points where histogram is maxed.
Recall that we defined xedges and yedges when we called hist2d.
Here, xedges contains the values of the bin edges along the 
x-axis (i.e., log(Ms)), and yedges contains the
values of the bin edges along the y-axis
(i.e., log(SFR)).

In [13]:
"""
pseudo code for the loop:

for each column do:

    save the midpoint of the column

    find the index of the row 
    that contains the most galaxies,
    call it hmax
    
    save the midpoint of the row that
    corresponds to hmax.
    
end do
"""

for i in range(0, len(xedges)-1):
    fMs[i]=(xedges[i]+xedges[i+1])/2.
    hmax=np.argmax(goodhist[i,:])
    fSFR[i]=(yedges[hmax]+yedges[hmax+1])/2.

In [ ]:
#let's check if the modes that were found are reasonable

plt.scatter(fMs,fSFR)
plt.xlabel('log($M_*$ [$M_\odot$])',fontsize=12)
plt.ylabel('log(SFR [$M_\odot$/yr])',fontsize=12)

Now, we want to fit a line to this plot.

First, impose reasonable limits overwhich the main sequence appear linear. Pull the values of fMs and fSFR where this is the case.

In [15]:
wh=np.nonzero((fMs>8.25) &(fMs<10.))
masked_fMs = fMs[wh]
masked_fSFR = fSFR[wh]

Next, define the equation for a line that we will be using with scipy's curve_fit routine (see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)

In [16]:
#define a function to fit to
def line(x,A,B): 
    return(A*x+B)

Use scipy.curve_fit to find the best-fit parameters to the line

In [17]:
popt, pcov = curve_fit(line, masked_fMs, masked_fSFR)

Lastly, estimate the error of the slope from the covariance and print it.

In [ ]:
perr = np.sqrt(np.diag(pcov))

slope=popt[0]
slopeerr=perr[0]
#print the main sequence slope
print('slope: ',slope)
print('error: ', slopeerr)

## Plotting it all together

We will now plot this line over the log(SFR) vs. log(Ms) plane.

In [ ]:
#Get the values of logmass and logSFR where logmass is finite.
#If you want, you can impose a fixed sSFR requirement. 
#A good choice is sSFR=10^-12
plt.hist2d(logmass[np.isfinite(logSFR)],\
           logSFR[np.isfinite(logSFR)], \
           bins=100, cmin=1)

#Plot the line that we fit
plt.plot(logmass, \
         line(logmass, *popt),\
         color='magenta')

#Add a colorbar and label it.
cbar=plt.colorbar()
cbar.set_label('$N_{\mathrm{galaxies}}$', rotation=90,fontsize=12)

#Label the figure axes
plt.xlabel('log($M_*$ [$M_\odot$])',fontsize=12)
plt.ylabel('log(SFR [$M_\odot$/yr])',fontsize=12)

#Edit the tick labels
plt.tick_params(axis='both', direction='in', \
                width=1.25,labelsize='medium', \
                which='both',top=True,right=True)

## Exercise: Use a different calculation of stellar mass or SFR to test the impact on the resulting main sequence slope

So far, we have used instantaneous star formation rates. Let's try repeating this experiment using the star formation that has occurred in the past 10 Myrs.

We already defined SFR_10 above from the galaxy catalog that we made in Notebook 1. Notice that SFR_10 has the same shape as the instantaneous SFRs that we were using, there are just zero values where no star formation has taken place in the past 10 Myrs.

In [ ]:
print(SFR_i.shape)
print(SFR_10.shape)

Now, let's define logSFR_10. We can use the same logMs array that we have been using, so we do not need to redefine that.

In [ ]:
#ignore the runetimewarning.
logSFR_10 = np.log10(SFR_10)

In the block below, plot the logSFR_10 vs. logMs plane

In [22]:
#Use hist2d to plot logSFR_10 vs logMs.
#Don't forget to either filter out the 
#non-finite SFRs or impose a fixed cutoff.

#Hint: copy the code for the first plot
#that we made and change
#logSFR to logSFR_10





Now, reference the code above and perform the ridge line fit to these data.

In [23]:
#Perform the ridge line fit to the new data below




Compare the slope of the line that you got here with the slope that we derived earlier. Are they statistically different?

Lastly, replot logSFR_10 vs. logMs and overlay the line that you just fit. How does this plot compare to the one that we made earler? 

In [24]:
#Replot logSFR_10 vs. logMs with the
#line that you fit above.
#If you'd like, change the x and y limits of this plot
#to match the one that we made earlier.
#You can do this with plt.ylim(lower_bound, upper_bound)
#and plt.xlim(lower_bound, upper_bound)





## Extension: Explore the dependence of the SFMS on timescale or aperture

Going back to Notebook 1, the SFR supplementary catalog contains SFRs averaged over different timescales and within different apertures. You can add those to your galaxy catalog and follow the procedure above to find the main sequence slope. Then, you can explore the dependence of the slope on these choices. You can also check out the results in Donnari+ (2019): https://arxiv.org/abs/1812.07584 or Pillepich+ (2019): https://arxiv.org/abs/1902.05553.

In [ ]:
### Extension ###

## Exercise: categorizing galaxies into star burst, main sequence, green valley, and quenched

The logSFR vs. logMs plane is often used to define different galaxy populations. These definitions usually categorize a galaxy as being a star burst, main sequence, green valley, or quench galaxy based on how far above or below the galaxy is from the main sequence ridge. 

In this exercise, you will classify galaxies based on the following definitions:

0. quenched: galSFR <= MS_SFR-1.1

1. green valley: MS_SFR-0.5 >= galSFR > MS_SFR-1.1

2. main sequence: MS_SFR+0.5 >= galSFR > MS_SFR-0.5

3. star burst: galSFR > MS_SFR.
    
Here, galSFR is the star formation of a galaxy (in dex) and MS_SFR is the corresponding value of the SFR along the main sequence ridge (in dex).

In the following code block, you will define a function that takes in an arbitrary list of stellar masses and SFRs (each of length N) and outputs an array galClassification (length N) that classifies the galaxies into one of the above categories. That is, each galaxy will be assigned a number (either 0, 1, 2, or 3) based on its classification.

Let's do this using the total stellar masses and the instantaneous star formation rates. Earlier, we determined the ridge of the line going through these data had a slope of 0.9059 and a y-intercept of -8.977.

In [25]:
def SF_cat(galMs, galSFR, MS_slope, MS_inter):
    """
    Params
    --------
    galMs - (N,) - array of galaxy stellar masses (in dex), 
                   value must be finite.
    galSFR - (N,) - array of galaxy star formation rates (in dex),
                    value must be finite.
    MS_slope - int - slope of the main sequence ridge.
    MS_inter - int - intercept of the main sequence ridge.
    
    Returns
    --------
    galClass - (N,) - classification of each galaxy
                            either 0, 1, 2, or 3 corresponding to
                            quenched, green valley, main sequence,
                            or star burst, respectively.
                            
    """
    
    #instantiate classification
    galClass = np.empty(len(galMs))
    
    #Define the values of MS_SFR.
    #That is, the values of the main sequence ridge 
    #corresponding to the stellar masses that 
    #we are interested in
    
    MS_SFR = #fill in the rest of this equation

    # Next, loop over each galaxy and classify it
    for i in range(len(galMs)):
        # Fill in the logic for each if statement
    
        if :#Fill in this logic
            galClass[i] = 0
            
        if :#Fill in this logic
            galClass[i] = 1
            
        if :#Fill in this logic
            galClass[i] = 2
            
        if :#Fill in this logic
            galClass[i] = 3
            
    return galClass

Now, let's remove the non-finite values from logSFR and logmass and classify these galaxies using the relation that we calculated above.

In [26]:
#Remove non-finite values
finite_logmass = logmass[np.isfinite(logSFR)]
finite_logSFR = logSFR[np.isfinite(logSFR)]

In [27]:
#classify the galaxies

### Can update this code to replace the slope and intercept with the appropriate variables ####
galClass = SF_cat(finite_logmass, finite_logSFR, 0.9059, -8.977)

Let's see how many of each galaxy type we have in our sample.

In [ ]:
print("There are ", len(finite_logSFR), " galaxies total")
print("There are ", len(galClass[galClass==0]), " quenched galaxies")
print("There are ", len(galClass[galClass==1]), " green valley galaxies")
print("There are ", len(galClass[galClass==2]), " main sequence galaxies")
print("There are ", len(galClass[galClass==3]), " star burst galaxies")

For reference, when the code was run for TNG100 with the default mass cut in Notebook 1, there were 35911 total galaxies, 2531 quenched galaxies, 5100 green valley galaxies, 15352 main sequence galaxies, and 12928 star burst galaxies.